# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

directory_path='../../05_src/data/prices'
PRICE_DATA = os.getenv("PRICE_DATA")


from glob import glob

parquet_files = glob(os.path.join(directory_path, "**/*.0.parquet"),recursive=True)
len(parquet_files)

11207

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
import dask.dataframe as dd
# I will read the parquet files and see the table below for a succesfull completion
dd_px = dd.read_parquet(parquet_files)

In [5]:
dd_px.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463043,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04 00:00:00+00:00,40.142929,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05 00:00:00+00:00,37.652863,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07 00:00:00+00:00,39.237465,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000


In [6]:
#Let's check the table if everything work well
dd_px.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463043,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04 00:00:00+00:00,40.142929,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05 00:00:00+00:00,37.652863,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07 00:00:00+00:00,39.237465,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000


In [7]:
#Step 1: Add lags variables Close and Adj Close.
dd_px=dd_px.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1)))

C:\Users\Zekiye\AppData\Local\Temp\ipykernel_27168\3806333837.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px=dd_px.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1)))


In [8]:
# Step 2: Calculate returns based on Adjusted Close
dd_px['returns'] = (dd_px['Adj Close'] / dd_px['Adj_Close_lag']) - 1

# Step 3: Calculate the high-low range for each day
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

# Assign the resulting DataFrame to `dd_feat`
dd_feat = dd_px[['Date', 'Close', 'Adj Close', 'Adj_Close_lag', 'returns', 'hi_lo_range']]

In [9]:
dd_feat.head()

Price,Date,Close,Adj Close,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,
HUM,2014-01-02 00:00:00+00:00,102.839996,95.121124,NaN,NaN,1.579994
HUM,2014-01-03 00:00:00+00:00,101.760002,94.122208,95.121124,-0.010502,1.459999
HUM,2014-01-06 00:00:00+00:00,100.760002,93.197273,94.122208,-0.009827,2.419998
HUM,2014-01-07 00:00:00+00:00,100.550003,93.003036,93.197273,-0.002084,2.849998
HUM,2014-01-08 00:00:00+00:00,99.400002,91.939346,93.003036,-0.011437,1.710007


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [10]:
# Write your code below.

import pandas as pd

# Step 4: Convert Dask DataFrame to pandas DataFrame
df_feat = dd_feat.compute()

# Step 5: Add a 10-day rolling average return calculation
df_feat_rtn = (df_feat
               .groupby(['ticker'])
               .apply(lambda x: x.assign(ma_return=x['returns']
                                         .rolling(window=10).mean())))

In [13]:
df_feat_rtn.head(50)

Price                              Date      Close  Adj Close  Adj_Close_lag  \
ticker ticker                                                                  
A      A      2003-01-02 00:00:00+00:00  13.690987  11.553922            NaN   
       A      2003-01-03 00:00:00+00:00  13.626609  11.499595      11.553922   
       A      2003-01-06 00:00:00+00:00  14.277539  12.048918      11.499595   
       A      2003-01-07 00:00:00+00:00  14.148784  11.940257      12.048918   
       A      2003-01-08 00:00:00+00:00  13.462089  11.360753      11.940257   
       A      2003-01-09 00:00:00+00:00  13.998569  11.813491      11.360753   
       A      2003-01-10 00:00:00+00:00  14.234621  12.012701      11.813491   
       A      2003-01-13 00:00:00+00:00  14.034335  11.843675      12.012701   
       A      2003-01-14 00:00:00+00:00  14.241774  12.018735      11.843675   
       A      2003-01-15 00:00:00+00:00  13.505007  11.396976      12.018735   
       A      2003-01-16 00:00:00+00:00  13.361946  11.276244      11.396976   
       A      2003-01-17 00:00:00+00:00  12.911302  10.895941      11.276244   
       A      2003-01-21 00:00:00+00:00  12.653791  10.678629      10.895941   
       A      2003-01-22 00:00:00+00:00  12.460658  10.515642      10.678629   
       A      2003-01-23 00:00:00+00:00  12.424893  10.485455      10.515642   
       A      2003-01-24 00:00:00+00:00  11.945637  10.081009      10.485455   
       A      2003-01-27 00:00:00+00:00  11.609442   9.797292      10.081009   
       A      2003-01-28 00:00:00+00:00  12.017167  10.141376       9.797292   
       A      2003-01-29 00:00:00+00:00  12.188841  10.286248      10.141376   
       A      2003-01-30 00:00:00+00:00  11.545064   9.742963      10.286248   
       A      2003-01-31 00:00:00+00:00  11.788269   9.948204       9.742963   
       A      2003-02-03 00:00:00+00:00  11.795422   9.954244       9.948204   
       A      2003-02-04 00:00:00+00:00  11.659514   9.839547       9.954244   
       A      2003-02-05 00:00:00+00:00  11.673820   9.851621       9.839547   
       A      2003-02-06 00:00:00+00:00   8.769671   7.400789       9.851621   
       A      2003-02-07 00:00:00+00:00   8.190272   6.911828       7.400789   
       A      2003-02-10 00:00:00+00:00   8.390558   7.080850       6.911828   
       A      2003-02-11 00:00:00+00:00   8.619456   7.274020       7.080850   
       A      2003-02-12 00:00:00+00:00   8.891273   7.503409       7.274020   
       A      2003-02-13 00:00:00+00:00   8.905579   7.515481       7.503409   
       A      2003-02-14 00:00:00+00:00   8.934192   7.539630       7.515481   
       A      2003-02-18 00:00:00+00:00   9.084406   7.666398       7.539630   
       A      2003-02-19 00:00:00+00:00   8.962804   7.563775       7.666398   
       A      2003-02-20 00:00:00+00:00   9.012876   7.606030       7.563775   
       A      2003-02-21 00:00:00+00:00   9.620887   8.119134       7.606030   
       A      2003-02-24 00:00:00+00:00   9.291845   7.841453       8.119134   
       A      2003-02-25 00:00:00+00:00   9.427754   7.956150       7.841453   
       A      2003-02-26 00:00:00+00:00   9.155937   7.726761       7.956150   
       A      2003-02-27 00:00:00+00:00   9.370529   7.907858       7.726761   
       A      2003-02-28 00:00:00+00:00   9.442060   7.968224       7.907858   
       A      2003-03-03 00:00:00+00:00   9.313305   7.859565       7.968224   
       A      2003-03-04 00:00:00+00:00   9.191702   7.756943       7.859565   
       A      2003-03-05 00:00:00+00:00   9.320458   7.865602       7.756943   
       A      2003-03-06 00:00:00+00:00   9.341917   7.883711       7.865602   
       A      2003-03-07 00:00:00+00:00   9.577969   8.082911       7.883711   
       A      2003-03-10 00:00:00+00:00   9.248927   7.805232       8.082911   
       A      2003-03-11 00:00:00+00:00   8.984263   7.581886       7.805232   
       A      2003-03-12 00:00:00+00:00   8.876967   7.491336       7.581886   


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [12]:
# It was not strictly necessary to convert to a pandas DataFrame to calculate the moving average return; 
# it could indeed have been calculated directly within Dask. Performing this calculation in Dask offers 
# several benefits, particularly for larger datasets, as Dask is designed for out-of-core computation 
# and can handle data that doesn’t fit into memory.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.